In [ ]:
%JPP_Home_Sales_starter_code_colab.py

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


0% [Working]
            
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease

0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (185.1
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Waiting for headers] [W
                                                                               
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.lau

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [ ]:
# Show DataFrame
df.show()

In [ ]:
# Import date time functions
from pyspark.sql.functions import year

# Show the year for the date column
df.select(year(df["date"])).show()

In [ ]:
# Save the year as a new column
df = df.withColumn("year", year(df['date']))
df.show()

In [ ]:
# Check the current schema in the Spark session
current_schema = spark.conf.get("spark.sql.defaultCatalog")
print("Current Schema: ", current_schema)

In [ ]:
# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales")

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
price4_bed_query = """
SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY  year(date)
ORDER BY year(date)
"""

# Execute the query for 3-bedroom, 3-bathroom homes
price4_bed_query = spark.sql(price4_bed_query)
price4_bed_query.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
price3_bed_3_Bath_sold_by_year_query = """
SELECT date_built, year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built, year(date)
ORDER BY date_built
"""

# Execute the query for 3-bedroom, 3-bathroom homes
price3_bed_3_Bath_sold_by_year = spark.sql(price3_bed_3_Bath_sold_by_year_query)
price3_bed_3_Bath_sold_by_year.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query = """
SELECT date_built, year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
AND sqft_living >=2000 AND floors = 2
GROUP BY date_built, year(date)
ORDER BY date_built
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query = spark.sql(Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query)
Avg_Price_3_bed_3_Bath_2_Floor_2000sqft_query.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
Avg_Price_Rating_over350K_query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_Rating_over350K_query = spark.sql(Avg_Price_Rating_over350K_query)
Avg_Price_Rating_over350K_query.show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

Avg_Price_Rating_over350K_query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query for 3-bedroom, 3-bathroom homes
Avg_Price_Rating_over350K_query = spark.sql(Avg_Price_Rating_over350K_query)
Avg_Price_Rating_over350K_query.show()

print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "/path/to/output/home_sales_partitioned"

# Partition the data by the "date_built" field and save it in parquet format
df.write.partitionBy("date_built").mode("append").parquet(output_path)

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("path_to_parquet_file_or_directory")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data_table")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# Run the query to calculate the average price of a home per "view" rating with average home price >= $350,000
result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""")

result.show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")
